In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [67]:
import addict
import copy
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

plt.rcParams['text.usetex'] = False # Plotting the global model is much much faster with tex fonts turned off

In [3]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
operators = addict.Dict()
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
celeri.get_all_mesh_smoothing_matrices(meshes)

# Eliminate matrix entries for vertical displacments and tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

Calculating cutde partials for triangles:   0%|          | 0/1841 [00:00<?, ?it/s]

In [4]:
def plot_slip_distributions(mesh, slip_distribution_input, slip_distribution_estimated, suptitle_string):
    marker_size = 2.0
    plt.figure(figsize=(12, 7))

    plt.subplot(2, 2, 1)
    plt.title("input strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 2)
    plt.title("input dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 3)
    plt.title("estimated strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 4)
    plt.title("estimated dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.suptitle(suptitle_string)
    plt.show()


def get_synthetic_displacements(mesh, tri_linear_operator):
    """
    Prescribe dip-slip in a Gaussian pattern.
    """
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - np.mean(mesh.centroids[:, 0])
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - np.mean(mesh.centroids[:, 1])

    # Hardcoded northern Cascadia example that Jack suggested.
    # tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - 234.5
    # tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - 48.5

    tri_centroid_to_mesh_centroid_distance = np.sqrt(tri_centroid_to_mesh_lon ** 2 + tri_centroid_to_mesh_lat ** 2)
    dip_slip_distribution = np.exp(-(tri_centroid_to_mesh_centroid_distance / 1.0) ** 2.0)
    slip_distribution = np.zeros(2 * dip_slip_distribution.size)
    slip_distribution[1::2] = dip_slip_distribution # Dip slip only
    synthetic_displacements = tri_linear_operator @ slip_distribution
    return slip_distribution, synthetic_displacements


def show_modes(mesh, eigenvectors, suptitle_string):
    triangulation = tri.Triangulation(mesh.centroids[:, 0], mesh.centroids[:, 1])
    plt.figure(figsize=(14, 4))
    for i in range(10):
        plt.subplot(2, 10, i + 1)
        plt.tricontourf(triangulation, eigenvectors[:, i][0::2], cmap="spring")
        plt.tricontour(triangulation, eigenvectors[:, i][0::2], colors="k", linewidths=0.25)
        plt.gca().set_aspect("equal", adjustable="box")
        plt.xlim([230.0, 240.0])
        plt.ylim([37.5, 52.5])
        plt.xticks([])
        plt.yticks([])

        plt.subplot(2, 10, 10 + i + 1)
        plt.tricontourf(triangulation, eigenvectors[:, i][1::2], cmap="cool")
        plt.tricontour(triangulation, eigenvectors[:, i][1::2], colors="k", linewidths=0.25)
        plt.gca().set_aspect("equal", adjustable="box")
        plt.xlim([230.0, 240.0])
        plt.ylim([37.5, 52.5])
        plt.xticks([])
        plt.yticks([])
    plt.suptitle(suptitle_string)
    plt.show()

# Generate Guassian slip source and synthetic displacements
slip_distribution, synthetic_displacements = get_synthetic_displacements(meshes[0], tde_matrix)

In [5]:
# Slip estimation with np.linalg.lstsq
estimated_slip_distribution = np.linalg.lstsq(tde_matrix, synthetic_displacements, rcond=None)
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution[0], suptitle_string="np.linalg.lstsq - no smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Slip estimation with direct inverse no smoothing
estimated_slip_distribution = np.linalg.inv(tde_matrix.T @ tde_matrix) @ tde_matrix.T @ synthetic_displacements
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - no smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Slip estimation with direct inverse and smoothing matrix
smoothing_matrix = meshes[0].smoothing_matrix.toarray()
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))
synthetic_displacements_with_smoothing = np.hstack((synthetic_displacements, np.zeros(smoothing_matrix.shape[0]).T))
estimated_slip_distribution = np.linalg.inv(tde_and_smoothing_matrix.T @ tde_and_smoothing_matrix) @ tde_and_smoothing_matrix.T @ synthetic_displacements_with_smoothing
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - with smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Slip estimation with truncated SVD
U, S, V_transpose = np.linalg.svd(tde_matrix)

In [9]:
# The general inverse is:
print("tde_matrix.shape :", tde_matrix.shape)
print("V_transpose.shape :", V_transpose.shape)
print("S.shape :", S.shape)
print("U.shape :", U.shape)
print("synthetic_displacements.shape :", synthetic_displacements.shape)

S_matrix = np.zeros_like(tde_matrix.T)
S_matrix_diag_index = np.arange(0, S.size)
S_matrix[S_matrix_diag_index, S_matrix_diag_index] = 1 / S
print("S_matrix.shape :", S_matrix.shape)

n_eigenvectors = 50
eigenvector_weights = np.linalg.pinv(tde_matrix @ V_transpose[:, 0:n_eigenvectors]) @ synthetic_displacements
estimated_slip_truncated_svd = V_transpose[:, 0:n_eigenvectors] @ eigenvector_weights
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Truncated SVD - Right eigenvectors only")

# Try full SVD
estimated_slip_full_svd = V_transpose @ S_matrix @ U.T @ synthetic_displacements
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Full SVD")



# Try the singular value approach for old times sake:
# https://stackoverflow.com/questions/18452633/how-do-i-associate-which-singular-value-corresponds-to-what-entry

tde_matrix.shape : (3372, 3682)
V_transpose.shape : (3682, 3682)
S.shape : (3372,)
U.shape : (3372, 3372)
synthetic_displacements.shape : (3372,)
S_matrix.shape : (3682, 3372)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Fall 2020 smoothing matrix SVD approach
# I think we only need the SVD of sparse smoothing matrix (I think there's scipy function for this)
import scipy

from scipy.sparse.linalg import svds

def delete_rows_and_columns_from_sparse_csr_matrix(mat, row_indices=[], col_indices=[]):
    """
    Lightly modified from:
    https://stackoverflow.com/questions/13077527/is-there-a-numpy-delete-equivalent-for-sparse-matrices
    Remove the rows (denoted by ``row_indices``) and columns (denoted by ``col_indices``) from the CSR sparse matrix ``mat``.
    WARNING: Indices of altered axes are reset in the returned matrix
    """
    # if not isinstance(mat, csr_matrix):
    #     raise ValueError("works only for CSR format -- use .tocsr() first")

    rows = []
    cols = []
    if any(row_indices):
        rows = list(row_indices)
    if any(col_indices):
        cols = list(col_indices)

    if len(rows) > 0 and len(cols) > 0:
        row_mask = np.ones(mat.shape[0], dtype=bool)
        row_mask[rows] = False
        col_mask = np.ones(mat.shape[1], dtype=bool)
        col_mask[cols] = False
        return mat[row_mask][:,col_mask]
    elif len(rows) > 0:
        mask = np.ones(mat.shape[0], dtype=bool)
        mask[rows] = False
        return mat[mask]
    elif len(cols) > 0:
        mask = np.ones(mat.shape[1], dtype=bool)
        mask[cols] = False
        return mat[:,mask]
    else:
        return mat


def scaled_sigmoid(arr, scalefactor):
    arr = 1 / (1 + np.exp(-arr * scalefactor)) - 0.5
    arr = 2 * arr #  Scale 0-1
    return arr


# Eliminate matrix entries for vertical displacments and tensile slip
smoothing_matrix = copy.deepcopy(meshes[0].smoothing_matrix)
smoothing_matrix = smoothing_matrix.tocsr()
rows_to_delete = np.arange(2, smoothing_matrix.shape[0], 3)
columns_to_delete = np.arange(2, smoothing_matrix.shape[1], 3)
smoothing_matrix = delete_rows_and_columns_from_sparse_csr_matrix(smoothing_matrix, rows_to_delete, columns_to_delete)

# Loveless style resolution scaling
resolution_vector = np.sum(tde_matrix ** 2, axis=0).T
resolution_vector_scaled = scaled_sigmoid(resolution_vector, 1e6)

n_eigenvalues = 200
# U, S, VT = scipy.sparse.linalg.svds(smoothing_matrix / resolution_vector_scaled, k=n_eigenvalues)
U, S, VT = scipy.sparse.linalg.svds(smoothing_matrix, k=n_eigenvalues)

G = tde_matrix @ VT.T
m_laptw = np.linalg.pinv(G) @ synthetic_displacements # Solve for eigenfunction weights
m_laptw_slip = VT.T @ m_laptw # Recover slip from eigenfunction weights
plot_slip_distributions(meshes[0], slip_distribution, m_laptw_slip, suptitle_string="Sparse Laplacian")

plt.figure()
plt.plot(S, "r+")
plt.show()


# eigenvector_weights = np.linalg.pinv(tde_matrix @ V_transpose[:, 0:n_eigenvectors]) @ synthetic_displacements
# estimated_slip_truncated_svd = V_transpose[:, 0:n_eigenvectors] @ eigenvector_weights
# plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Truncated SVD - Right eigenvectors only")

# % Eigenfunctions, unweighted
# [Vt, Et] = eigs(w, neigs, 0);

# % Eigenfunctions, weighted by resolution
# [Vtw, Etw] = eigs(w./resvec, neigs, 0);

# % Loveless style resolution
# resolution_loveless = sum(T.^2, 1)';
# resvec = scaledsigmoid(resolution_loveless, 1e6);

# % % Unweighted
# % m_lapt = (T * Vt) \ d; % Solve for eigenfunction weights
# % lambda = 1e-3;
# % %m_lapt = [T*Vt ; lambda*eye(i)] \ [d_noise(:) ; zeros(i,1)];
# % m_lapt_slip = Vt * m_lapt; % Recover slip from eigenfunction weights

# % Weighted
# G = T * Vtw;
# m_laptw = G \ d; % Solve for eigenfunction weights
# % m_laptw = [T*Vtw ; lambda*eye(i)] \ [d_noise(:) ; zeros(i,1)];
# m_laptw_slip = Vtw * m_laptw; % Recover slip from eigenfunction weights
# m_est = m_laptw_slip;


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# U, S, VT = np.linalg.svd(smoothing_matrix.toarray())
# U, S, VT = np.linalg.eig(smoothing_matrix.toarray())

n_eigenvalues = 20

w, VT = scipy.sparse.linalg.eigsh(smoothing_matrix, n_eigenvalues)
plt.figure()
plt.plot(w, "r+")
plt.show()

eigenvector_weights = np.linalg.pinv(tde_matrix @ VT[:, 0:n_eigenvectors]) @ synthetic_displacements
estimated_slip_truncated_svd = VT[:, 0:n_eigenvectors] @ eigenvector_weights
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Truncated SVD - Right eigenvectors only")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Try visualizating some of the eigenvectors

U, S, VT = np.linalg.svd(smoothing_matrix.toarray())


eigenvector_weights = np.linalg.pinv(tde_matrix @ VT[:, 0:n_eigenvectors]) @ synthetic_displacements
estimated_slip_truncated_svd = VT[:, 0:n_eigenvectors] @ eigenvector_weights
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Truncated SVD - Right eigenvectors only")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
VT[:, 0:n_eigenvectors]
plt.figure()
plt.title("estimated dip-slip")
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=VT[:, -1][1::2], alpha=0.5)
plt.colorbar()
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def get_tri_smoothing_matrix_simple(share):
    """
    Produces a smoothing matrix based on the scale-dependent
    umbrella operator (e.g., Desbrun et al., 1999; Resor, 2004).

    Inputs:
    share: n x 3 array of indices of the up to 3 elements sharing a side
        with each of the n elements

    Outputs:
    smoothing matrix: 3n x 3n smoothing matrix
    """

    # Allocate sparse matrix for contructing smoothing matrix
    n_shared_tris = share.shape[0]
    smoothing_matrix = scipy.sparse.lil_matrix((3 * n_shared_tris, 3 * n_shared_tris))

    # Place the weights into the smoothing operator
    for j in range(3):
        for i in range(n_shared_tris):
            smoothing_matrix[3 * i + j, 3 * i + j] = 3
            if share[i, j] != -1:
                k = 3 * i + np.array([0, 1, 2])
                m = 3 * share[i, j] + np.array([0, 1, 2])
                smoothing_matrix[k, m] = -1
    return smoothing_matrix

share = celeri.get_shared_sides(meshes[0].verts)
smoothing_matrix_simple = get_tri_smoothing_matrix_simple(share)

# smoothing_matrix = copy.deepcopy(meshes[0].smoothing_matrix)
smoothing_matrix_simple = smoothing_matrix_simple.tocsr()
rows_to_delete = np.arange(2, smoothing_matrix_simple.shape[0], 3)
columns_to_delete = np.arange(2, smoothing_matrix_simple.shape[1], 3)
smoothing_matrix_simple = delete_rows_and_columns_from_sparse_csr_matrix(smoothing_matrix_simple, rows_to_delete, columns_to_delete)



# Eigenvectors from: np.linalg.svd

In [111]:
_, _, right_eigenvectors = np.linalg.svd(smoothing_matrix_simple.toarray())
smallest_right_eigenvectors = right_eigenvectors[-11:-1, :].T
show_modes(meshes[0], smallest_right_eigenvectors, r"""np.linalg.svd(smoothing_matrix_simple.toarray())""")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Eigenvectors from: scipy.sparse.linalg.eigsh

whichstr [‘LM’ | ‘SM’ | ‘LA’ | ‘SA’ | ‘BE’]
If A is a complex Hermitian matrix, ‘BE’ is invalid. Which k eigenvectors and eigenvalues to find:

‘LM’ : Largest (in magnitude) eigenvalues.

‘SM’ : Smallest (in magnitude) eigenvalues.

‘LA’ : Largest (algebraic) eigenvalues.

‘SA’ : Smallest (algebraic) eigenvalues.

‘BE’ : Half (k/2) from each end of the spectrum.

In [109]:
_, right_eigenvectors = scipy.sparse.linalg.eigsh(smoothing_matrix_simple, n_eigenvalues, which="LM")
show_modes(meshes[0], right_eigenvectors, r"""scipy.sparse.linalg.eigsh(smoothing_matrix, n_eigenvalues, which="LM")""")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
_, right_eigenvectors = scipy.sparse.linalg.eigsh(smoothing_matrix_simple, n_eigenvalues, which="SM")
show_modes(meshes[0], right_eigenvectors, r"""scipy.sparse.linalg.eigsh(smoothing_matrix, n_eigenvalues, which="SM")""")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
_, right_eigenvectors = scipy.sparse.linalg.eigsh(smoothing_matrix_simple, n_eigenvalues, which="LA")
show_modes(meshes[0], right_eigenvectors, r"""scipy.sparse.linalg.eigsh(smoothing_matrix, n_eigenvalues, which="LA")""")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [106]:
_, right_eigenvectors = scipy.sparse.linalg.eigsh(smoothing_matrix_simple, n_eigenvalues, which="SA")
show_modes(meshes[0], right_eigenvectors, r"""scipy.sparse.linalg.eigsh(smoothing_matrix, n_eigenvalues, which="SA")""")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
_, right_eigenvectors = scipy.sparse.linalg.eigsh(smoothing_matrix_simple, n_eigenvalues, which="BE")
show_modes(meshes[0], right_eigenvectors, r"""scipy.sparse.linalg.eigsh(smoothing_matrix, n_eigenvalues, which="BE")""")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …